# Dependencies

In [1]:
pip install --upgrade mxnet~=1.7 gluonts

     |████████████████████████████████| 46.9MB 65kB/s 
     |████████████████████████████████| 901kB 41.5MB/s 
     |████████████████████████████████| 10.1MB 24.6MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
import numpy as np
import sklearn


/usr/local/lib/python3.7/dist-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


# DataFrame Preprocessing

In [4]:
df=pd.read_csv('hd_gm-2.csv')

In [5]:
df.head()

,STR_ALIAS,date_snap,weekly_sales_amt,weekly_shrink_amount,ty_net_shrink,ty_inv_sls,inv_cnt,cycle_length,FET_FLG,LU_FLG,urban,suburban,rural,rgn_nm,us_unemployment_rate_16_24,us_unemployment_rate,fyfw_prd_bgn,fyfw_prd_end,str_fet_wk_cnt,net_sls_amt,rtn_amt_rt,rtn_trans_rt,mkup_rt,rsn_01_rt,rsn_02_rt,dc_dmg_rt,str_dmg_rt,cust_sat_rt,rsn_06_rt,str_use_rt,rsn_09_rt,rdtag_rt,rsn_12_rt,rsn_15_rt,rsn_18_rt,rsn_22_rt,rsn_23_rt,rsn_24_rt,rsn_25_rt,promo_rt,...,inval_unt_intrvntn_rt,cash_pen,sco_pen,key_sku_rt,qty_key_rt,gen_sku_rt,d90_cash_vs_total,d90_sco_vs_total,d90_cash_vs_main,d90_sco_vs_main,attr_rt,quick_term_rt,avg_tenure,avg_wage,pt_ratio,int_cnt,int_val,ext_cnt,ext_val,known_theft_cnt,known_theft_val,merch_recov_cnt,merch_recov_val,pos_recov_cnt,pos_recov_val,risk_avt_cnt,risk_avt_val,lowes_flag,lowes_cnt,menards_flag,menards_cnt,avg_osha_rt,aci_score,cap_index,online_sat_voc_num,online_sat_voc_den,online_sat_voc_pct,zip_code,shrink_rate,state
0,1001,10/2/17,556736.78,10426.41,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201635,201735,0,22038030.28,-0.149609,0.101869,0.009331,0.000324,0.000773,0.000211,0.007403,0.004279,0.000131,0.000503,0.009140,0.001536,0.001531,0.001433,0.000005,0.000058,0.001622,0.000765,0.001998,0.007829,...,0.25065,0.282240,0.110415,0.059294,0.034961,0.000064,0.418338,0.155321,0.729245,0.270755,0.010618,0.208333,5.412885,12.605000,0.633269,2,1844.75,7,7407.0,3,1800.0,18,12339.19,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,164.08,193,0.850155,30096,0.018728,GA
1,1001,10/9/17,557228.39,10435.62,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201636,201736,0,22559095.42,-0.150751,0.101917,0.009346,0.000317,0.000774,0.000221,0.007390,0.004273,0.000128,0.000549,0.009109,0.001554,0.001549,0.001440,0.000005,0.000057,0.001656,0.000755,0.002017,0.007692,...,0.25065,0.282114,0.110415,0.059323,0.034963,0.000069,0.418329,0.155359,0.729192,0.270808,0.010769,0.205479,5.411538,12.604808,0.633269,2,1844.75,7,7407.0,3,1800.0,18,12339.19,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,164.23,193,0.850933,30096,0.018728,GA
2,1001,10/16/17,599390.85,11225.23,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201637,201737,0,23073264.01,-0.151623,0.102010,0.009298,0.000313,0.000762,0.000221,0.007410,0.004266,0.000128,0.000548,0.009008,0.001568,0.001563,0.001447,0.000005,0.000055,0.001655,0.000755,0.002045,0.007553,...,0.25065,0.282093,0.110415,0.059364,0.034979,0.000068,0.419431,0.155190,0.729926,0.270074,0.010324,0.214286,5.407885,12.603462,0.633654,2,1844.75,7,7407.0,2,1042.0,17,11909.38,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,163.67,193,0.848031,30096,0.018728,GA
3,1001,10/23/17,549696.03,10294.56,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201638,201738,0,23634939.47,-0.151075,0.102063,0.009369,0.000319,0.000751,0.000216,0.007434,0.004222,0.000127,0.000549,0.008942,0.001559,0.001554,0.001446,0.000005,0.000072,0.001672,0.000741,0.002087,0.007423,...,0.25065,0.282017,0.110415,0.059451,0.034974,0.000064,0.419698,0.155206,0.730031,0.269969,0.010174,0.217391,5.406538,12.603269,0.634038,2,1844.75,7,7407.0,3,1301.0,18,12106.27,1,239.85,2,814.72,1,5,0,0,0.815405,0,0,162.96,191,0.853194,30096,0.018728,GA
4,1001,10/30/17,639228.97,11971.31,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201639,201739,0,24155476.07,-0.151058,0.102077,0.009518,0.000380,0.000751,0.000220,0.007457,0.004198,0.000124,0.000551,0.008897,0.001535,0.001531,0.001463,0.000004,0.000071,0.001683,0.000735,0.002087,0.007440,...,0.25065,0.282124,0.110415,0.059494,0.034875,0.000061,0.420342,0.156345,0.728891,0.271109,0.010172,0.217391,5.405962,12.603269,0.634038,2,1844.75,7,7407.0,3,1301.0,19,12305.27,1,239.85,2,814.72,1,5,0,0,0.815405,0,0,161.74,189,0.855767,30096,0.018728,GA


In [6]:
df['weekly_shrink_amount'] = df['weekly_sales_amt'] * df['shrink_rate']

In [12]:
df = df.round({'weekly_shrink_amount': 2})

In [13]:
df.to_csv('hd_gm_fixed.csv', index=False, encoding='utf-8')

In [14]:
df = df.drop(['STR_ALIAS'], axis=1)

# 1 STR / Region

In [3]:
df = pd.read_csv('hd_gm_fixed.csv')

In [10]:
rgns = [df.rgn_nm.unique()]

In [11]:
rgns

[array(['MIDSOUTH', 'SOUTHEAST', 'SOUTH FLORIDA/PR', 'SOUTHWEST',
        'PAC MTN DESERT', 'GULF', 'PACIFIC SOUTH', 'PACIFIC NORTH',
        'PACIFIC CENTRAL', 'SOUTH ATLANTIC', 'NJ METRO', 'NY METRO',
        'OHIO VALLEY', 'PACIFIC NORTHWEST', 'MID-ATLANTIC', 'CENTRAL',
        'MIDWEST', 'NORTHERN PLAINS', 'NEW ENGLAND'], dtype=object)]

In [9]:
pd.options.display.max_rows = 4000

In [4]:
dict_of_regions = dict(iter(df.groupby('rgn_nm')))


In [10]:
dict_of_regions


{'CENTRAL':        STR_ALIAS date_snap  weekly_sales_amt  ...  zip_code  shrink_rate  state
 33956       1695    2/6/17         401262.10  ...     62226     0.013224     IL
 33957       1695   2/13/17         443399.25  ...     62226     0.013224     IL
 33958       1695   2/20/17         511582.18  ...     62226     0.013224     IL
 33959       1695   2/27/17         516735.14  ...     62226     0.013224     IL
 33960       1695    3/6/17         472573.62  ...     62226     0.013224     IL
 ...          ...       ...               ...  ...       ...          ...    ...
 96771       2984   6/17/19         402019.86  ...     63005     0.027808     MO
 96772       2984   6/24/19         402019.86  ...     63005     0.027808     MO
 96773       2984    7/1/19         402019.86  ...     63005     0.027808     MO
 96774       2984    7/8/19         402019.86  ...     63005     0.027808     MO
 96775       2984   7/15/19         402019.86  ...     63005     0.027808     MO
 
 [4910 rows x 1

In [ ]:
midsouth = df[df.rgn_nm == 'MIDSOUTH']

In [7]:
df.head()

,STR_ALIAS,date_snap,weekly_sales_amt,weekly_shrink_amount,ty_net_shrink,ty_inv_sls,inv_cnt,cycle_length,FET_FLG,LU_FLG,urban,suburban,rural,rgn_nm,us_unemployment_rate_16_24,us_unemployment_rate,fyfw_prd_bgn,fyfw_prd_end,str_fet_wk_cnt,net_sls_amt,rtn_amt_rt,rtn_trans_rt,mkup_rt,rsn_01_rt,rsn_02_rt,dc_dmg_rt,str_dmg_rt,cust_sat_rt,rsn_06_rt,str_use_rt,rsn_09_rt,rdtag_rt,rsn_12_rt,rsn_15_rt,rsn_18_rt,rsn_22_rt,rsn_23_rt,rsn_24_rt,rsn_25_rt,promo_rt,...,inval_unt_intrvntn_rt,cash_pen,sco_pen,key_sku_rt,qty_key_rt,gen_sku_rt,d90_cash_vs_total,d90_sco_vs_total,d90_cash_vs_main,d90_sco_vs_main,attr_rt,quick_term_rt,avg_tenure,avg_wage,pt_ratio,int_cnt,int_val,ext_cnt,ext_val,known_theft_cnt,known_theft_val,merch_recov_cnt,merch_recov_val,pos_recov_cnt,pos_recov_val,risk_avt_cnt,risk_avt_val,lowes_flag,lowes_cnt,menards_flag,menards_cnt,avg_osha_rt,aci_score,cap_index,online_sat_voc_num,online_sat_voc_den,online_sat_voc_pct,zip_code,shrink_rate,state
0,1001,10/2/17,556736.78,10426.57,611648.84,32660070.09,3.0,55.0,0.0,1.0,0.0,1.0,0.0,MIDSOUTH,8.7,4.1,201635.0,201735.0,0.0,22038030.28,-0.149609,0.101869,0.009331,0.000324,0.000773,0.000211,0.007403,0.004279,0.000131,0.000503,0.009140,0.001536,0.001531,0.001433,0.000005,0.000058,0.001622,0.000765,0.001998,0.007829,...,0.25065,0.282240,0.110415,0.059294,0.034961,0.000064,0.418338,0.155321,0.729245,0.270755,0.010618,0.208333,5.412885,12.605000,0.633269,2.0,1844.75,7.0,7407.0,3.0,1800.0,18.0,12339.19,0.0,0.00,2.0,814.72,1.0,5.0,0.0,0.0,0.815405,0.0,0.0,164.08,193.0,0.850155,30096.0,0.018728,GA
1,1001,10/9/17,557228.39,10435.77,611648.84,32660070.09,3.0,55.0,0.0,1.0,0.0,1.0,0.0,MIDSOUTH,8.7,4.1,201636.0,201736.0,0.0,22559095.42,-0.150751,0.101917,0.009346,0.000317,0.000774,0.000221,0.007390,0.004273,0.000128,0.000549,0.009109,0.001554,0.001549,0.001440,0.000005,0.000057,0.001656,0.000755,0.002017,0.007692,...,0.25065,0.282114,0.110415,0.059323,0.034963,0.000069,0.418329,0.155359,0.729192,0.270808,0.010769,0.205479,5.411538,12.604808,0.633269,2.0,1844.75,7.0,7407.0,3.0,1800.0,18.0,12339.19,0.0,0.00,2.0,814.72,1.0,5.0,0.0,0.0,0.815405,0.0,0.0,164.23,193.0,0.850933,30096.0,0.018728,GA
2,1001,10/16/17,599390.85,11225.39,611648.84,32660070.09,3.0,55.0,0.0,1.0,0.0,1.0,0.0,MIDSOUTH,8.7,4.1,201637.0,201737.0,0.0,23073264.01,-0.151623,0.102010,0.009298,0.000313,0.000762,0.000221,0.007410,0.004266,0.000128,0.000548,0.009008,0.001568,0.001563,0.001447,0.000005,0.000055,0.001655,0.000755,0.002045,0.007553,...,0.25065,0.282093,0.110415,0.059364,0.034979,0.000068,0.419431,0.155190,0.729926,0.270074,0.010324,0.214286,5.407885,12.603462,0.633654,2.0,1844.75,7.0,7407.0,2.0,1042.0,17.0,11909.38,0.0,0.00,2.0,814.72,1.0,5.0,0.0,0.0,0.815405,0.0,0.0,163.67,193.0,0.848031,30096.0,0.018728,GA
3,1001,10/23/17,549696.03,10294.71,611648.84,32660070.09,3.0,55.0,0.0,1.0,0.0,1.0,0.0,MIDSOUTH,8.7,4.1,201638.0,201738.0,0.0,23634939.47,-0.151075,0.102063,0.009369,0.000319,0.000751,0.000216,0.007434,0.004222,0.000127,0.000549,0.008942,0.001559,0.001554,0.001446,0.000005,0.000072,0.001672,0.000741,0.002087,0.007423,...,0.25065,0.282017,0.110415,0.059451,0.034974,0.000064,0.419698,0.155206,0.730031,0.269969,0.010174,0.217391,5.406538,12.603269,0.634038,2.0,1844.75,7.0,7407.0,3.0,1301.0,18.0,12106.27,1.0,239.85,2.0,814.72,1.0,5.0,0.0,0.0,0.815405,0.0,0.0,162.96,191.0,0.853194,30096.0,0.018728,GA
4,1001,10/30/17,639228.97,11971.48,611648.84,32660070.09,3.0,55.0,0.0,1.0,0.0,1.0,0.0,MIDSOUTH,8.7,4.1,201639.0,201739.0,0.0,24155476.07,-0.151058,0.102077,0.009518,0.000380,0.000751,0.000220,0.007457,0.004198,0.000124,0.000551,0.008897,0.001535,0.001531,0.001463,0.000004,0.000071,0.001683,0.000735,0.002087,0.007440,...,0.25065,0.282124,0.110415,0.059494,0.034875,0.000061,0.420342,0.156345,0.728891,0.271109,0.010172,0.217391,5.405962,12.603269,0.634038,2.0,1844.75,7.0,7407.0,3.0,1301.0,19.0,12305.27,1.0,239.85,2.0,814.72,1.0,5.0,0.0,0.0,0.815405,0.0,0.0,161.74,189.0,0.855767,30096.0,0.018728,GA


# Global Model

In [3]:
df_global = pd.read_csv('hd_gm_fixed.csv')

In [4]:
df_global.head()

,STR_ALIAS,date_snap,weekly_sales_amt,weekly_shrink_amount,ty_net_shrink,ty_inv_sls,inv_cnt,cycle_length,FET_FLG,LU_FLG,urban,suburban,rural,rgn_nm,us_unemployment_rate_16_24,us_unemployment_rate,fyfw_prd_bgn,fyfw_prd_end,str_fet_wk_cnt,net_sls_amt,rtn_amt_rt,rtn_trans_rt,mkup_rt,rsn_01_rt,rsn_02_rt,dc_dmg_rt,str_dmg_rt,cust_sat_rt,rsn_06_rt,str_use_rt,rsn_09_rt,rdtag_rt,rsn_12_rt,rsn_15_rt,rsn_18_rt,rsn_22_rt,rsn_23_rt,rsn_24_rt,rsn_25_rt,promo_rt,...,inval_unt_intrvntn_rt,cash_pen,sco_pen,key_sku_rt,qty_key_rt,gen_sku_rt,d90_cash_vs_total,d90_sco_vs_total,d90_cash_vs_main,d90_sco_vs_main,attr_rt,quick_term_rt,avg_tenure,avg_wage,pt_ratio,int_cnt,int_val,ext_cnt,ext_val,known_theft_cnt,known_theft_val,merch_recov_cnt,merch_recov_val,pos_recov_cnt,pos_recov_val,risk_avt_cnt,risk_avt_val,lowes_flag,lowes_cnt,menards_flag,menards_cnt,avg_osha_rt,aci_score,cap_index,online_sat_voc_num,online_sat_voc_den,online_sat_voc_pct,zip_code,shrink_rate,state
0,1001,10/2/17,556736.78,10426.57,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201635,201735,0,22038030.28,-0.149609,0.101869,0.009331,0.000324,0.000773,0.000211,0.007403,0.004279,0.000131,0.000503,0.009140,0.001536,0.001531,0.001433,0.000005,0.000058,0.001622,0.000765,0.001998,0.007829,...,0.25065,0.282240,0.110415,0.059294,0.034961,0.000064,0.418338,0.155321,0.729245,0.270755,0.010618,0.208333,5.412885,12.605000,0.633269,2,1844.75,7,7407.0,3,1800.0,18,12339.19,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,164.08,193,0.850155,30096,0.018728,GA
1,1001,10/9/17,557228.39,10435.77,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201636,201736,0,22559095.42,-0.150751,0.101917,0.009346,0.000317,0.000774,0.000221,0.007390,0.004273,0.000128,0.000549,0.009109,0.001554,0.001549,0.001440,0.000005,0.000057,0.001656,0.000755,0.002017,0.007692,...,0.25065,0.282114,0.110415,0.059323,0.034963,0.000069,0.418329,0.155359,0.729192,0.270808,0.010769,0.205479,5.411538,12.604808,0.633269,2,1844.75,7,7407.0,3,1800.0,18,12339.19,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,164.23,193,0.850933,30096,0.018728,GA
2,1001,10/16/17,599390.85,11225.39,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201637,201737,0,23073264.01,-0.151623,0.102010,0.009298,0.000313,0.000762,0.000221,0.007410,0.004266,0.000128,0.000548,0.009008,0.001568,0.001563,0.001447,0.000005,0.000055,0.001655,0.000755,0.002045,0.007553,...,0.25065,0.282093,0.110415,0.059364,0.034979,0.000068,0.419431,0.155190,0.729926,0.270074,0.010324,0.214286,5.407885,12.603462,0.633654,2,1844.75,7,7407.0,2,1042.0,17,11909.38,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,163.67,193,0.848031,30096,0.018728,GA
3,1001,10/23/17,549696.03,10294.71,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201638,201738,0,23634939.47,-0.151075,0.102063,0.009369,0.000319,0.000751,0.000216,0.007434,0.004222,0.000127,0.000549,0.008942,0.001559,0.001554,0.001446,0.000005,0.000072,0.001672,0.000741,0.002087,0.007423,...,0.25065,0.282017,0.110415,0.059451,0.034974,0.000064,0.419698,0.155206,0.730031,0.269969,0.010174,0.217391,5.406538,12.603269,0.634038,2,1844.75,7,7407.0,3,1301.0,18,12106.27,1,239.85,2,814.72,1,5,0,0,0.815405,0,0,162.96,191,0.853194,30096,0.018728,GA
4,1001,10/30/17,639228.97,11971.48,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201639,201739,0,24155476.07,-0.151058,0.102077,0.009518,0.000380,0.000751,0.000220,0.007457,0.004198,0.000124,0.000551,0.008897,0.001535,0.001531,0.001463,0.000004,0.000071,0.001683,0.000735,0.002087,0.007440,...,0.25065,0.282124,0.110415,0.059494,0.034875,0.000061,0.420342,0.156345,0.728891,0.271109,0.010172,0.217391,5.405962,12.603269,0.634038,2,1844.75,7,7407.0,3,1301.0,19,12305.27,1,239.85,2,814.72,1,5,0,0,0.815405,0,0,161.74,189,0.855767,30096,0.018728,GA


In [5]:
df_global['date_snap'] = pd.to_datetime(df_global['date_snap'], format='%m/%d/%y')


In [9]:
df_global = df_global.rename(columns={"STR_ALIAS": "id"})

In [10]:
df_global.tail()

,id,date_snap,weekly_sales_amt,weekly_shrink_amount,ty_net_shrink,ty_inv_sls,inv_cnt,cycle_length,FET_FLG,LU_FLG,urban,suburban,rural,rgn_nm,us_unemployment_rate_16_24,us_unemployment_rate,fyfw_prd_bgn,fyfw_prd_end,str_fet_wk_cnt,net_sls_amt,rtn_amt_rt,rtn_trans_rt,mkup_rt,rsn_01_rt,rsn_02_rt,dc_dmg_rt,str_dmg_rt,cust_sat_rt,rsn_06_rt,str_use_rt,rsn_09_rt,rdtag_rt,rsn_12_rt,rsn_15_rt,rsn_18_rt,rsn_22_rt,rsn_23_rt,rsn_24_rt,rsn_25_rt,promo_rt,...,inval_unt_intrvntn_rt,cash_pen,sco_pen,key_sku_rt,qty_key_rt,gen_sku_rt,d90_cash_vs_total,d90_sco_vs_total,d90_cash_vs_main,d90_sco_vs_main,attr_rt,quick_term_rt,avg_tenure,avg_wage,pt_ratio,int_cnt,int_val,ext_cnt,ext_val,known_theft_cnt,known_theft_val,merch_recov_cnt,merch_recov_val,pos_recov_cnt,pos_recov_val,risk_avt_cnt,risk_avt_val,lowes_flag,lowes_cnt,menards_flag,menards_cnt,avg_osha_rt,aci_score,cap_index,online_sat_voc_num,online_sat_voc_den,online_sat_voc_pct,zip_code,shrink_rate,state
96895,2987,2019-06-24,1135704.79,13426.30,736938.3,62338209.32,1,64,0,0,0,0,1,PACIFIC NORTHWEST,9.3,3.6,201821,201920,0,48611515.98,-0.136772,0.092312,0.010824,0.000789,0.000620,0.000036,0.004102,0.003937,0.000237,0.000406,0.021788,0.003314,0.003294,0.000232,0.000020,0.000304,0.001627,0.001549,0.001506,0.009464,...,0.298327,0.176200,0.122660,0.041297,0.034870,0.000124,0.351419,0.204830,0.631765,0.368235,0.007052,0.102041,3.730769,14.314423,0.546538,0,0.0,1,149.0,1,149.99,0,0.0,0,0.0,0,0.0,0,0,0,0,2.098034,0,0,587.88,691,0.850767,98382,0.011822,WA
96896,2987,2019-07-01,1267448.13,14983.77,736938.3,62338209.32,1,64,0,0,0,0,1,PACIFIC NORTHWEST,9.1,3.6,201822,201921,0,48710968.04,-0.136645,0.092245,0.010688,0.000791,0.000637,0.000040,0.004132,0.003979,0.000227,0.000406,0.021813,0.003324,0.003305,0.000237,0.000019,0.000248,0.001616,0.001542,0.001491,0.009437,...,0.298327,0.175839,0.122660,0.041177,0.034871,0.000125,0.351998,0.204254,0.632803,0.367197,0.006910,0.104167,3.731538,14.320962,0.545962,0,0.0,1,149.0,1,149.99,0,0.0,0,0.0,0,0.0,0,0,0,0,2.159741,0,0,588.54,692,0.850491,98382,0.011822,WA
96897,2987,2019-07-08,1130075.28,13359.75,736938.3,62338209.32,1,64,0,0,0,0,1,PACIFIC NORTHWEST,9.1,3.6,201823,201922,0,48762851.14,-0.136682,0.092308,0.010631,0.000792,0.000664,0.000042,0.004171,0.004008,0.000225,0.000410,0.021580,0.003412,0.003393,0.000230,0.000019,0.000252,0.001646,0.001527,0.001499,0.009428,...,0.298327,0.175413,0.122660,0.041162,0.034787,0.000125,0.353151,0.203885,0.633982,0.366018,0.007487,0.153846,3.733269,14.328846,0.545000,0,0.0,1,149.0,1,149.99,0,0.0,0,0.0,0,0.0,0,0,0,0,2.159741,0,0,590.26,697,0.846858,98382,0.011822,WA
96898,2987,2019-07-15,966585.43,11426.97,736938.3,62338209.32,1,64,0,0,0,0,1,PACIFIC NORTHWEST,9.1,3.6,201824,201923,0,48820818.87,-0.137618,0.092390,0.010584,0.000808,0.000655,0.000043,0.004226,0.004110,0.000218,0.000415,0.021819,0.003516,0.003497,0.000231,0.000020,0.000236,0.001672,0.001526,0.001508,0.009455,...,0.298327,0.174930,0.122660,0.041410,0.034758,0.000129,0.352505,0.204329,0.633053,0.366947,0.007775,0.148148,3.734231,14.336923,0.544231,0,0.0,1,149.0,1,149.99,0,0.0,0,0.0,0,0.0,0,0,0,0,2.159741,0,0,596.54,704,0.847358,98382,0.011822,WA
96899,2987,2019-07-22,1068878.43,12636.28,736938.3,62338209.32,1,64,0,0,0,0,1,PACIFIC NORTHWEST,9.1,3.6,201825,201924,0,48801495.17,-0.139975,0.092583,0.010710,0.000827,0.000671,0.000043,0.004389,0.004302,0.000215,0.000417,0.021781,0.003498,0.003476,0.000237,0.000022,0.000375,0.001693,0.001521,0.001758,0.009443,...,0.298327,0.174593,0.005428,0.041564,0.034799,0.000126,0.353976,0.203755,0.634672,0.365328,0.008063,0.142857,3.736731,14.345577,0.543077,0,0.0,1,149.0,1,149.99,0,0.0,0,0.0,0,0.0,0,0,0,0,2.159741,0,0,595.17,705,0.844213,98382,0.011822,WA


In [11]:
FORECAST_HORIZON = 26

In [13]:
data = df_global[['id', 'date_snap', 'weekly_shrink_amount']]

In [14]:
data.head()

,id,date_snap,weekly_shrink_amount
0,1001,2017-10-02,10426.57
1,1001,2017-10-09,10435.77
2,1001,2017-10-16,11225.39
3,1001,2017-10-23,10294.71
4,1001,2017-10-30,11971.48


# Store Model

In [ ]:
df = df.set_index('date_snap')

In [ ]:
df.head()

,weekly_sales_amt,weekly_shrink_amount,ty_net_shrink,ty_inv_sls,inv_cnt,cycle_length,FET_FLG,LU_FLG,urban,suburban,rural,rgn_nm,us_unemployment_rate_16_24,us_unemployment_rate,fyfw_prd_bgn,fyfw_prd_end,str_fet_wk_cnt,net_sls_amt,rtn_amt_rt,rtn_trans_rt,mkup_rt,rsn_01_rt,rsn_02_rt,dc_dmg_rt,str_dmg_rt,cust_sat_rt,rsn_06_rt,str_use_rt,rsn_09_rt,rdtag_rt,rsn_12_rt,rsn_15_rt,rsn_18_rt,rsn_22_rt,rsn_23_rt,rsn_24_rt,rsn_25_rt,promo_rt,rsn_30_rt,rsn_33_rt,...,inval_unt_intrvntn_rt,cash_pen,sco_pen,key_sku_rt,qty_key_rt,gen_sku_rt,d90_cash_vs_total,d90_sco_vs_total,d90_cash_vs_main,d90_sco_vs_main,attr_rt,quick_term_rt,avg_tenure,avg_wage,pt_ratio,int_cnt,int_val,ext_cnt,ext_val,known_theft_cnt,known_theft_val,merch_recov_cnt,merch_recov_val,pos_recov_cnt,pos_recov_val,risk_avt_cnt,risk_avt_val,lowes_flag,lowes_cnt,menards_flag,menards_cnt,avg_osha_rt,aci_score,cap_index,online_sat_voc_num,online_sat_voc_den,online_sat_voc_pct,zip_code,shrink_rate,state
date_snap,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-10-02,556736.78,10426.41,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201635,201735,0,22038030.28,-0.149609,0.101869,0.009331,0.000324,0.000773,0.000211,0.007403,0.004279,0.000131,0.000503,0.009140,0.001536,0.001531,0.001433,0.000005,0.000058,0.001622,0.000765,0.001998,0.007829,0.003467,-0.000272,...,0.25065,0.282240,0.110415,0.059294,0.034961,0.000064,0.418338,0.155321,0.729245,0.270755,0.010618,0.208333,5.412885,12.605000,0.633269,2,1844.75,7,7407.0,3,1800.0,18,12339.19,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,164.08,193,0.850155,30096,0.018728,GA
2017-10-09,557228.39,10435.62,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201636,201736,0,22559095.42,-0.150751,0.101917,0.009346,0.000317,0.000774,0.000221,0.007390,0.004273,0.000128,0.000549,0.009109,0.001554,0.001549,0.001440,0.000005,0.000057,0.001656,0.000755,0.002017,0.007692,0.003403,-0.000282,...,0.25065,0.282114,0.110415,0.059323,0.034963,0.000069,0.418329,0.155359,0.729192,0.270808,0.010769,0.205479,5.411538,12.604808,0.633269,2,1844.75,7,7407.0,3,1800.0,18,12339.19,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,164.23,193,0.850933,30096,0.018728,GA
2017-10-16,599390.85,11225.23,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201637,201737,0,23073264.01,-0.151623,0.102010,0.009298,0.000313,0.000762,0.000221,0.007410,0.004266,0.000128,0.000548,0.009008,0.001568,0.001563,0.001447,0.000005,0.000055,0.001655,0.000755,0.002045,0.007553,0.003382,-0.000270,...,0.25065,0.282093,0.110415,0.059364,0.034979,0.000068,0.419431,0.155190,0.729926,0.270074,0.010324,0.214286,5.407885,12.603462,0.633654,2,1844.75,7,7407.0,2,1042.0,17,11909.38,0,0.00,2,814.72,1,5,0,0,0.815405,0,0,163.67,193,0.848031,30096,0.018728,GA
2017-10-23,549696.03,10294.56,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201638,201738,0,23634939.47,-0.151075,0.102063,0.009369,0.000319,0.000751,0.000216,0.007434,0.004222,0.000127,0.000549,0.008942,0.001559,0.001554,0.001446,0.000005,0.000072,0.001672,0.000741,0.002087,0.007423,0.003405,-0.000269,...,0.25065,0.282017,0.110415,0.059451,0.034974,0.000064,0.419698,0.155206,0.730031,0.269969,0.010174,0.217391,5.406538,12.603269,0.634038,2,1844.75,7,7407.0,3,1301.0,18,12106.27,1,239.85,2,814.72,1,5,0,0,0.815405,0,0,162.96,191,0.853194,30096,0.018728,GA
2017-10-30,639228.97,11971.31,611648.84,32660070.09,3,55,0,1,0,1,0,MIDSOUTH,8.7,4.1,201639,201739,0,24155476.07,-0.151058,0.102077,0.009518,0.000380,0.000751,0.000220,0.007457,0.004198,0.000124,0.000551,0.008897,0.001535,0.001531,0.001463,0.000004,0.000071,0.001683,0.000735,0.002087,0.007440,0.003440,-0.000265,...,0.25065,0.282124,0.110415,0.059494,0.034875,0.000061,0.420342,0.156345,0.728891,0.271109,0.010172,0.217391,5.405962,12.603269,0.634038,2,1844.75,7,7407.0,3,1301.0,19,12305.27,1,239.85,2,814.72,1,5,0,0,0.815405,0,0,161.74,189,0.855767,30096,0.018728,GA


In [ ]:
df.tail()

,weekly_sales_amt,weekly_shrink_amount,ty_net_shrink,ty_inv_sls,inv_cnt,cycle_length,FET_FLG,LU_FLG,urban,suburban,rural,rgn_nm,us_unemployment_rate_16_24,us_unemployment_rate,fyfw_prd_bgn,fyfw_prd_end,str_fet_wk_cnt,net_sls_amt,rtn_amt_rt,rtn_trans_rt,mkup_rt,rsn_01_rt,rsn_02_rt,dc_dmg_rt,str_dmg_rt,cust_sat_rt,rsn_06_rt,str_use_rt,rsn_09_rt,rdtag_rt,rsn_12_rt,rsn_15_rt,rsn_18_rt,rsn_22_rt,rsn_23_rt,rsn_24_rt,rsn_25_rt,promo_rt,rsn_30_rt,rsn_33_rt,...,inval_unt_intrvntn_rt,cash_pen,sco_pen,key_sku_rt,qty_key_rt,gen_sku_rt,d90_cash_vs_total,d90_sco_vs_total,d90_cash_vs_main,d90_sco_vs_main,attr_rt,quick_term_rt,avg_tenure,avg_wage,pt_ratio,int_cnt,int_val,ext_cnt,ext_val,known_theft_cnt,known_theft_val,merch_recov_cnt,merch_recov_val,pos_recov_cnt,pos_recov_val,risk_avt_cnt,risk_avt_val,lowes_flag,lowes_cnt,menards_flag,menards_cnt,avg_osha_rt,aci_score,cap_index,online_sat_voc_num,online_sat_voc_den,online_sat_voc_pct,zip_code,shrink_rate,state
date_snap,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-08-24,851887.43,32685.98,1347095.52,35109052.01,3,48,0,1,0,1,0,MIDSOUTH,14.4,8.4,201929,202029,0,36317459.70,-0.152146,0.100839,0.013255,0.000442,0.000844,0.000246,0.007531,0.004010,0.000160,0.001021,0.013805,0.001080,0.000999,0.001210,0.000080,0.000080,0.002761,0.001528,0.002107,0.007898,0.002639,-0.000651,...,0.255709,0.208748,0.282856,0.060924,0.031918,0.000051,0.422041,0.223670,0.653607,0.346393,0.005712,0.142857,6.389500,13.743500,0.544000,1,33.35,1,2000.0,2,2000.0,0,0.0,0,0.0,0,0.0,1,5,0,0,0.0,0,0,245.23,334,0.734222,30096,0.038369,GA
2020-08-31,912187.46,34999.62,1347095.52,35109052.01,3,48,0,1,0,1,0,MIDSOUTH,14.4,8.4,201930,202030,0,36507570.68,-0.152451,0.100924,0.013376,0.000442,0.000843,0.000243,0.007599,0.003984,0.000160,0.001010,0.013637,0.001028,0.000950,0.001196,0.000079,0.000079,0.002771,0.001472,0.002170,0.007894,0.002638,-0.000628,...,0.250830,0.207258,0.290103,0.060938,0.031918,0.000053,0.423561,0.222673,0.655430,0.344570,0.005587,0.153846,6.402632,13.748421,0.543684,1,33.35,1,2000.0,2,2000.0,0,0.0,0,0.0,0,0.0,1,5,0,0,0.0,0,0,250.19,340,0.735853,30096,0.038369,GA
2020-09-07,897739.57,34445.28,1347095.52,35109052.01,3,48,0,1,0,1,0,MIDSOUTH,13.5,7.8,201931,202031,0,36733128.76,-0.152338,0.100840,0.013616,0.000445,0.000835,0.000240,0.007567,0.004007,0.000161,0.001013,0.013508,0.001009,0.000941,0.001177,0.000068,0.000111,0.002798,0.001567,0.002212,0.007842,0.002606,-0.000604,...,0.251830,0.205830,0.297902,0.061005,0.032021,0.000050,0.424799,0.222312,0.656454,0.343546,0.005901,0.153846,6.416667,13.754444,0.543333,1,33.35,1,2000.0,2,2000.0,0,0.0,0,0.0,0,0.0,1,5,0,0,0.0,0,0,249.71,341,0.732287,30096,0.038369,GA
2020-09-14,870249.27,33390.50,1347095.52,35109052.01,3,48,0,1,0,1,0,MIDSOUTH,13.5,7.8,201932,202032,0,36959614.55,-0.151820,0.100975,0.013401,0.000446,0.000825,0.000240,0.007618,0.004007,0.000159,0.001000,0.013620,0.000968,0.000901,0.001163,0.000067,0.000110,0.002767,0.001598,0.002184,0.007882,0.002617,-0.000580,...,0.251812,0.204668,0.305499,0.060948,0.032141,0.000051,0.425610,0.222091,0.657108,0.342892,0.006256,0.153846,6.434118,13.762353,0.542941,1,33.35,1,2000.0,2,2000.0,0,0.0,0,0.0,0,0.0,1,5,0,0,0.0,0,0,249.89,340,0.734971,30096,0.038369,GA
2020-09-21,815786.88,31300.84,1347095.52,35109052.01,3,48,0,1,0,1,0,MIDSOUTH,13.5,7.8,201933,202033,0,37149976.44,-0.151903,0.100931,0.013225,0.000438,0.000822,0.000251,0.007670,0.004000,0.000157,0.001004,0.013616,0.000941,0.000877,0.001154,0.000064,0.000109,0.002762,0.001620,0.002225,0.007869,0.002591,-0.000571,...,0.252622,0.203508,0.313118,0.060938,0.032192,0.000063,0.426981,0.221708,0.658222,0.341778,0.006141,0.083333,6.449375,13.770000,0.542500,1,33.35,1,2000.0,2,2000.0,0,0.0,0,0.0,0,0.0,1,5,0,0,0.0,0,0,246.08,335,0.734567,30096,0.038369,GA


In [ ]:
freq="1week"
start_train = pd.Timestamp("2017-10-02 00:15:00", freq=freq)
start_test = pd.Timestamp("2019-10-07 05:30:00", freq=freq)
prediction_lentgh=672